In [1]:
#importing libraries
import pandas as pd
import yfinance as yf
from datetime import datetime

In the below code, we first imported the necessary Python libraries and downloaded the historical stock price data for four companies: Apple, Microsoft, Netflix, and Google, for the last three months.

In [2]:
#create the time frame of the data -- 3 months ago from now
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

In [3]:
#Selecting the company for analysis by ticker symbol
tickers = ['AAPL','MSFT','NFLX','GOOG']

In [4]:
#create an empty list and append the ticker symbols
df_list =[]

for ticker in tickers:
    data=yf.download(ticker,start=start_date,end=end_date)
    df_list.append(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
#concatenate the ticker symbols with the relevant date
df = pd.concat(df_list,keys=tickers,names =['ticker','Date'])

In [8]:
#see the first 10 rows
print(df.head(10))

                         Open        High         Low       Close   Adj Close  \
ticker Date                                                                     
AAPL   2023-03-20  155.070007  157.820007  154.149994  157.399994  157.182587   
       2023-03-21  157.320007  159.399994  156.539993  159.279999  159.059982   
       2023-03-22  159.300003  162.139999  157.809998  157.830002  157.612000   
       2023-03-23  158.830002  161.550003  157.679993  158.929993  158.710464   
       2023-03-24  158.860001  160.339996  157.850006  160.250000  160.028656   
       2023-03-27  159.940002  160.770004  157.869995  158.279999  158.061371   
       2023-03-28  157.970001  158.490005  155.979996  157.649994  157.432236   
       2023-03-29  159.369995  161.050003  159.350006  160.770004  160.547943   
       2023-03-30  161.529999  162.470001  161.270004  162.360001  162.135742   
       2023-03-31  162.440002  165.000000  161.910004  164.899994  164.672226   

                     Volume

In [9]:
df = df.reset_index()
print(df.head())

  ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-03-20  155.070007  157.820007  154.149994  157.399994   
1   AAPL 2023-03-21  157.320007  159.399994  156.539993  159.279999   
2   AAPL 2023-03-22  159.300003  162.139999  157.809998  157.830002   
3   AAPL 2023-03-23  158.830002  161.550003  157.679993  158.929993   
4   AAPL 2023-03-24  158.860001  160.339996  157.850006  160.250000   

    Adj Close    Volume  
0  157.182587  73641400  
1  159.059982  73938300  
2  157.612000  75701800  
3  158.710464  67622100  
4  160.028656  59196500  


In [15]:
#plotting the chart performance
import plotly.express as px

fig =px.line(df, x ='Date',y='Close',color ='ticker',title="Stock Market Performance for the Last 3 Months")

fig.show()

In [16]:
#comparing perfomance via charts
fig = px.area(df,x = 'Date',y='Close',color ='ticker',facet_col ='ticker',labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},title='Stock Prices for Apple, Microsoft, Netflix, and Google')

fig.show()


In [18]:
#Analysing the moving average as way to analyse trends and pattersn over the 3 months
df['MA10'] = df.groupby('ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
58  179.752002  176.529002
59  180.353001  177.091001
60  181.023001  177.685001
61  181.615001  178.351000
62  182.012001  178.844500

[63 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
189         NaN         NaN
190         NaN         NaN
191         NaN         NaN
192         NaN         NaN
193         NaN         NaN
..          ...         ...
247  124.498001  123.543000
248  124.477001  123.916500
249  124.578001  124.131001
250  124.720000  124.346500
251  124.603000  124.373500

[63 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
..          ...         ...
121  330.448001  324.28

In [23]:
#visualize the moving averages for the stocks
for ticker,group in df.groupby('ticker'):
    fig= px.line(group,x='Date', y=['Close','MA10','MA20'],title = f"{ticker} Moving Averages")

fig.show()

The output shows four separate graphs for each company. When the MA10 crosses above the MA20, it is considered a bullish signal indicating that the stock price will continue to rise. Conversely, when the MA10 crosses below the MA20, it is a bearish signal that the stock price will continue falling.

In [24]:
#Measuring volatility
df['Volatility'] = df.groupby('ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='ticker', 
              title='Volatility of All Companies')
fig.show()

High volatility indicates that the stock or market experiences large and frequent price movements, while low volatility indicates that the market experiences smaller or less frequent price movements.

In [26]:
import statsmodels


In [27]:

#Correlation --AAPL & MSFT
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()

There is a strong linear relationship between the stock prices of Apple and Microsoft, which means that when the stock price of Apple increases, the stock price of Microsoft also tends to increase. It is a sign of a strong correlation or similarity between the two companies, which can be due to factors such as industry trends, market conditions, or common business partners or customers. For investors, this positive correlation may indicate an opportunity to diversify their portfolio by investing in both companies, as both stocks may offer similar potential returns and risks.

So this is how you can perform stock market analysis using Python.